### Prepare the environment

In [27]:
import pandas as pd
from gensim.models.phrases import Phrases
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_numeric
import re
from gensim import utils
from gensim.corpora.dictionary import Dictionary
from gensim.models import ldamodel
import pyLDAvis.gensim_models as gensim_models

In [28]:
# removed "_" from regular expression
punctuation = r"""!"#$%&'()*+,-./:;<=>?@[\]^`{|}~"""
RE_PUNCT = re.compile(r'([%s])+' % re.escape(punctuation), re.UNICODE)


def strip_punctuation(s):
    s = utils.to_unicode(s)
    return RE_PUNCT.sub(" ", s)

### Import dataset

In [29]:
df_text = pd.read_excel(open('data/diffusion_innovation_open_ended.xls','rb'), sheet_name='diffusion_innovation_open_ended')

### Knowledge topic model

In [30]:
knowledge_stream = df_text['knowledge'].tolist()
#Preprocess
my_filter = [
    lambda x: x.lower(), strip_tags, strip_punctuation,
    strip_multiple_whitespaces, strip_numeric,
    remove_stopwords, strip_short]

bigram = Phrases(knowledge_stream, min_count=1, threshold=2)
knowledge_bigrams = ["".join(bigram[i]) for i in knowledge_stream]
knowledge_clean = [preprocess_string(i, filters=my_filter) for i in knowledge_bigrams]

print(knowledge_clean[0:5])    

[['share', 'insights'], ['space', 'central', 'person', 'access', 'useful', 'topics', 'trainings', 'accessed'], ['share', 'examples', 'works'], ['continuous', 'exposure', 'upskilling', 'training'], ['spread', 'responsibility', 'great', 'effort', 'upskilling', 'obtaining', 'buy', 'non', 'departments']]


In [31]:
dct_knowledge = Dictionary(knowledge_clean)
corpus = [dct_knowledge.doc2bow(doc) for doc in knowledge_clean]
print(len(dct_knowledge))
print(len(corpus))

435
142


In [32]:
lda = ldamodel.LdaModel(corpus, id2word=dct_knowledge, num_topics = 5)
lda.show_topics(num_words=10)

[(0,
  '0.094*"training" + 0.016*"people" + 0.013*"knowledge" + 0.013*"technology" + 0.012*"sessions" + 0.012*"workshops" + 0.012*"new" + 0.012*"experience" + 0.012*"create" + 0.012*"skills"'),
 (1,
  '0.048*"training" + 0.035*"data" + 0.031*"knowledge" + 0.026*"sharing" + 0.019*"employees" + 0.015*"teams" + 0.013*"share" + 0.013*"platforms" + 0.012*"work" + 0.012*"sessions"'),
 (2,
  '0.057*"training" + 0.034*"people" + 0.016*"awareness" + 0.012*"data" + 0.012*"technology" + 0.008*"employees" + 0.008*"best" + 0.008*"ideas" + 0.008*"use" + 0.008*"volunteers"'),
 (3,
  '0.023*"sharing" + 0.020*"tools" + 0.020*"product" + 0.019*"knowledge" + 0.017*"data" + 0.016*"need" + 0.015*"sessions" + 0.012*"plan" + 0.009*"training" + 0.009*"skills"'),
 (4,
  '0.027*"training" + 0.025*"data" + 0.021*"skills" + 0.021*"knowledge" + 0.016*"forums" + 0.014*"business" + 0.014*"having" + 0.014*"online" + 0.014*"trainings" + 0.013*"use"')]

In [38]:
# vis = gensim_models.prepare(lda, corpus, dct_knowledge)
# pyLDAvis.save_html(vis, 'knowledge_lda.html')




import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dct_knowledge)
vis



/home/d5mit/anaconda3/envs/data-driven-org/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/home/d5mit/anaconda3/envs/data-driven-org/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/d5mit/anaconda3/envs/data-driven-org/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/d5mit/anaconda3/envs/data-driven-org/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  f

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.018259 -0.091349       1        1  20.841965
3      0.129228  0.059520       2        1  20.818938
0     -0.056831  0.034462       3        1  20.195537
2     -0.087879  0.047855       4        1  19.264667
4     -0.002778 -0.050489       5        1  18.878893, topic_info=             Term       Freq      Total Category  logprob  loglift
14       training  39.000000  39.000000  Default  30.0000  30.0000
29         people  10.000000  10.000000  Default  29.0000  29.0000
122       sharing   9.000000   9.000000  Default  28.0000  28.0000
186       product   4.000000   4.000000  Default  27.0000  27.0000
159          need   3.000000   3.000000  Default  26.0000  26.0000
..            ...        ...        ...      ...      ...      ...
78   organization   0.842371   3.262841   Topic5  -5.2496   0.3130
147          base   0.842187   1.977977   Topic5  -5.2499   0.8133
154        attend   0.841697   1.977798   Topic5  -5.2504   0.8128
275      maturity   0.841416   2.006261   Topic5  -5.2508   0.7982
262      specific   0.841399   2.721000   Topic5  -5.2508   0.4934

[304 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
333       3  0.368121       able
333       5  0.736243       able
2         1  0.373528     access
2         2  0.747056     access
266       5  0.732978   achieves
...     ...       ...        ...
34        5  0.137430       work
302       2  0.213678  workshops
302       3  0.427355  workshops
302       4  0.213678  workshops
302       5  0.213678  workshops

[330 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3, 5])

### Persuasion topic model

In [39]:
persuasion_stream = df_text['persuasion'].tolist()
print(persuasion_stream[0:10])
#Preprocess
my_filter = [
    lambda x: x.lower(), strip_tags, strip_punctuation,
    strip_multiple_whitespaces, strip_numeric,
    remove_stopwords, strip_short]

#bigram = Phrases(persuasion_stream, min_count=1, threshold=2)
#persuasion_bigrams = ["".join(bigram[i]) for i in persuasion_stream]
persuasion_clean = [preprocess_string(i, filters=my_filter) for i in persuasion_stream]

print(persuasion_clean[0:5])   

['Lead by example, do not lead on opinion', 'Describe the benefits and how it can lead to more informed decisions and the accuracy using a data driven approach rather than gut-feeling or based on misinterpreted analysis.', 'Share examples of how it works', 'Introduction presentations and demos of data driven technologies', 'Selling the value and efficiency that it brings. This might require somebody working closely with the dept to understand were the value add lies.', 'training  workshops', 'Awards/Certificates for participation and different levels of competency. Possible implementation of ideas brought about through the contribution of the participants.', 'Highlight competitive advantage, highlight the existence of trends hidden in data, highlight the criticality of decisions based on trusted information, highlight the need to have an appetite for change (willingness to change direction based on observations within the data).', 'have a prominent display section in for instance the d

TypeError: decoding to str: need a bytes-like object, float found